# 1. Import thư viện

In [1]:
import cv2
import numpy as np
import os
from PIL import Image
import shutil
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from torch.nn.parallel import DataParallel
from torch.cuda.amp import autocast, GradScaler
import torch.nn.functional as F
from torchvision.utils import save_image

from tqdm import tqdm


from models.sr_model import *
from models.wdsr import *
from models.srresnet_ import *
from models.sr_model import *
from models.utils import *
from models.vdsr import *
from models.srcnn import *
import time

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Số lượng tham số của mô hình: 664704


In [2]:
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:128'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 2. Tạo Mô hình SR

In [3]:
# edsr_srcnn = EDSR_srcnnnnfy()

# srresnet = SRResNet().to(device)
# srcnn = RCAN().to(device)
vdsr = VDSR().to(device)
srcnn = SRCNN().to(device)


In [4]:

# # edsr_srcnna.load_state_dict(torch.load('weight/best_edsrx4_model.pth', map_location=device, weights_only=True))
# edsr_orig.load_state_dict(torch.load('weight/best_edsrx4_orig_model.pth', map_location=device))
# edrn_sobel.load_state_dict(torch.load('weight/best_sobel_srx4_model.pth', map_location=device))
# edrn_canny.load_state_dict(torch.load('weight/best_canny_srx4_model.pth', map_location=device))
# srresnet.load_state_dict(torch.load('weight/model_srresnet.pth', map_location=device))
# srcnn.load_state_dict(torch.load('weight/srcnn_x4.pth', map_location=device))
# vdsr.load_state_dict(torch.load('weight/vdsr_x4.pth', map_location=device))


In [5]:
class ImageDataset(Dataset):
    def __init__(self, lr_dir, hr_dir):
        self.lr_files = sorted(os.listdir(lr_dir))
        self.hr_files = sorted(os.listdir(hr_dir))
        self.lr_dir = lr_dir
        self.hr_dir = hr_dir

    def __len__(self):
        return len(self.lr_files)

    def __getitem__(self, idx):
        lr_image = cv2.imread(os.path.join(self.lr_dir, self.lr_files[idx]))
        hr_image = cv2.imread(os.path.join(self.hr_dir, self.hr_files[idx]))
        lr_image = cv2.cvtColor(lr_image, cv2.COLOR_BGR2RGB)
        hr_image = cv2.cvtColor(hr_image, cv2.COLOR_BGR2RGB)
        hr_height, hr_width = hr_image.shape[:2]
        # scale_factor = 4
        # lr_height, lr_width = height // 2, width // 2
        # hr_height, hr_width = height * 2, width * 2

        # # Rescale ảnh LR lên 4 lần
        # lr_image = cv2.resize(lr_image, (lr_width, lr_height), interpolation=cv2.INTER_CUBIC)
        lr_image = cv2.resize(lr_image, (hr_width, hr_height), interpolation=cv2.INTER_CUBIC)
        
        # hr_image = cv2.resize(hr_image, (hr_width, hr_height), interpolation=cv2.INTER_CUBIC)

        transform = transforms.Compose([
            transforms.ToPILImage(),
            transforms.ToTensor()
        ])
        
        lr_image = transform(lr_image)
        hr_image = transform(hr_image)
        return lr_image, hr_image
    
# class ImageDataset(Dataset):
    # def __init__(self, lr_dir, hr_dir):
    #     self.lr_files = sorted(os.listdir(lr_dir))
    #     self.hr_files = sorted(os.listdir(hr_dir))
    #     self.lr_dir = lr_dir
    #     self.hr_dir = hr_dir

    # def __len__(self):
    #     return len(self.lr_files)

    # def __getitem__(self, idx):
    #     lr_image = Image.open(os.path.join(self.lr_dir, self.lr_files[idx])).convert('RGB')
    #     hr_image = Image.open(os.path.join(self.hr_dir, self.hr_files[idx])).convert('RGB')

    #     width, height = lr_image.size
        
    #     # Tính kích thước mới bằng cách nhân với scale_factor
    #     new_width = width * 4
    #     new_height = height * 4

    #     # Resize ảnh LR lên theo kích thước mới
    #     lr_image = lr_image.resize((new_width, new_height), Image.BICUBIC)

    #     hr_image = np.array(hr_image).astype(np.float32)
    #     lr_image = np.array(lr_image).astype(np.float32)
    #     hr_image = convert_rgb_to_y(hr_image)
    #     lr_image = convert_rgb_to_y(lr_image)
    #     hr_image = np.expand_dims(hr_image / 255., 0)
    #     lr_image = np.expand_dims(lr_image / 255., 0)
    #     transform = transforms.Compose([
    #         # transforms.ToPILImage(),
    #         transforms.ToTensor()
    #     ])
        
    #     # lr_image = transform(lr_image)
    #     # hr_image = transform(hr_image)
    #     return lr_image, hr_image

# 3. Tạo Hyperparameter

In [6]:
# Đường dẫn tới bộ dữ liệu
train_lr_dir = 'Train/LR'
train_hr_dir = 'Train/HR'
valid_lr_dir = 'Test/LR'
valid_hr_dir = 'Test/HR'
# test_hr_dir  = '/kaggle/input/srdataset/sr_data/test/HR'
# test_lr_dir  = '/kaggle/input/srdataset/sr_data/test/LR'

# print(torch.cuda.memory_allocated())
# print(torch.cuda.memory_reserved())

In [7]:
from torch.amp import autocast, GradScaler
scaler = GradScaler()

# Khởi tạo dataset và dataloader
train_dataset = ImageDataset(train_lr_dir, train_hr_dir)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

valid_dataset = ImageDataset(valid_lr_dir, valid_hr_dir)
valid_loader = DataLoader(valid_dataset)

# vdsr_train_dataset = vdsrImageDataset(train_lr_dir, train_hr_dir)
# vdsr_train_loader = DataLoader(vdsr_train_dataset, batch_size = 16, shuffle=True)

# vdsr_valid_dataset = vdsrImageDataset(valid_lr_dir, valid_hr_dir)
# vdsr_valid_loader = DataLoader(vdsr_valid_dataset)
print(len(train_loader))

# Khởi tạo loss function
criterion = nn.MSELoss()

# Khởi tạo optimizers, schedulers cho từng mô hình
# optim_edsr = optim.Adam(edsr_orig.parameters(), lr=1e-4, betas=(0.9, 0.999))
# scheduler_edsr = optim.lr_scheduler.StepLR(optim_edsr, step_size=10**5, gamma=0.5)

# optim_srresnet = optim.Adam(srresnet.parameters(), lr=1e-4, betas=(0.9, 0.999))
# scheduler_srresnet = optim.lr_scheduler.StepLR(optim_srresnet, step_size=10**5, gamma=0.5)

optim_srcnn = optim.Adam(srcnn.parameters(), lr=1e-4, betas=(0.9, 0.999))
scheduler_srcnn = optim.lr_scheduler.StepLR(optim_srcnn, step_size=10**5, gamma=0.5)

optim_vdsr = optim.Adam(vdsr.parameters(), lr=1e-4, betas=(0.9, 0.999))
scheduler_vdsr = optim.lr_scheduler.StepLR(optim_vdsr, step_size=10**5, gamma=0.5)

12500


In [8]:
def calculate_psnr(img1, img2):
    mse = torch.mean((img1 - img2) ** 2)
    if mse == 0:
        return float('inf')
    max_pixel = 1.0
    psnr = 20 * torch.log10(max_pixel / torch.sqrt(mse))
    return psnr.item()

In [9]:
print(device)

cuda


# 4. Training

In [10]:
# Hàm tính PSNR
def calculate_psnr(img1, img2):
    mse = torch.mean((img1 - img2) ** 2)
    if mse == 0:
        return float('inf')
    max_pixel = 1.0
    psnr = 20 * torch.log10(max_pixel / torch.sqrt(mse))
    return psnr.item()

num_epochs = 24

best_psnr_srcnn = float('-inf')
best_psnr_vdsr = float('-inf')
torch.cuda.empty_cache()

losses_srcnn = []
losses_vdsr = []

avg_psnr_srcnn = []
avg_psnr_vdsr = []

val_avg_psnr_srcnn = []
val_avg_psnr_vdsr = []

patience = 50
epochs_no_improve = 0
log_file = open('training_log_models_srcnnn_vdsr.txt', 'a')

for epoch in range(num_epochs):
    srcnn.train()
    vdsr.train()

    epoch_loss_srcnn, psnr_values_srcnn = 0, 0
    epoch_loss_vdsr, psnr_values_vdsr = 0, 0
    start_time = time.time()

    # Training loop for srcnn
    for (lr_images, hr_images) in tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}', unit='batch'):
        lr_images = lr_images.cuda()
        hr_images = hr_images.cuda()

        # Train srcnn model
        optim_srcnn.zero_grad()
        with autocast(device_type='cuda'):
            outputs_srcnn =srcnn(lr_images)
            loss_srcnn = criterion(outputs_srcnn, hr_images)
        psnr_srcnn = calculate_psnr(outputs_srcnn, hr_images)

        scaler.scale(loss_srcnn).backward()
        scaler.step(optim_srcnn)
        scaler.update()
        scheduler_srcnn.step()

        epoch_loss_srcnn += loss_srcnn.item()
        psnr_values_srcnn += psnr_srcnn

        optim_vdsr.zero_grad()
        with autocast(device_type='cuda'):
            outputs_vdsr = vdsr(lr_images)
            loss_vdsr = criterion(outputs_vdsr, hr_images)
        psnr_vdsr = calculate_psnr(outputs_vdsr, hr_images)

        scaler.scale(loss_vdsr).backward()
        scaler.step(optim_vdsr)
        scaler.update()
        scheduler_vdsr.step()

        epoch_loss_vdsr += loss_vdsr.item()
        psnr_values_vdsr += psnr_vdsr
      
    # Training loop for vdsr
   
        
    # Average losses and PSNRs
    avg_epoch_loss_srcnn = epoch_loss_srcnn / len(train_loader)
    avg_psnr_srcnn_epoch = psnr_values_srcnn / len(train_loader)
    losses_srcnn.append(avg_epoch_loss_srcnn)
    avg_psnr_srcnn.append(avg_psnr_srcnn_epoch)

    avg_epoch_loss_vdsr = epoch_loss_vdsr / len(train_loader)
    avg_psnr_vdsr_epoch = psnr_values_vdsr / len(train_loader)
    losses_vdsr.append(avg_epoch_loss_vdsr)
    avg_psnr_vdsr.append(avg_psnr_vdsr_epoch)

    # Validation for srcnn and vdsr
    srcnn.eval()
    vdsr.eval()

    val_psnr_srcnn, val_psnr_vdsr = 0, 0

    with torch.no_grad():
        # Validate srcnn
        for (lr_images, hr_images) in valid_loader:
            lr_images = lr_images.cuda()
            hr_images = hr_images.cuda()

            outputs_srcnn = srcnn(lr_images)
            psnr_srcnn = calculate_psnr(outputs_srcnn, hr_images)
            val_psnr_srcnn += psnr_srcnn


            outputs_vdsr = vdsr(lr_images)
            psnr_vdsr = calculate_psnr(outputs_vdsr, hr_images)
            val_psnr_vdsr += psnr_vdsr
        # Validate vdsr
        


    val_avg_psnr_srcnn_epoch = val_psnr_srcnn / len(valid_loader)
    val_avg_psnr_srcnn.append(val_avg_psnr_srcnn_epoch)

    val_avg_psnr_vdsr_epoch = val_psnr_vdsr / len(valid_loader)
    val_avg_psnr_vdsr.append(val_avg_psnr_vdsr_epoch)

    # Save best model for srcnn
    if val_avg_psnr_srcnn_epoch > best_psnr_srcnn:
        best_psnr_srcnn = val_avg_psnr_srcnn_epoch
        torch.save(srcnn.state_dict(), 'best_wdsra.pth')
        print(f"Saved SRCNN model with PSNR {best_psnr_srcnn:.4f}")
    # Save best model for vdsr
    if val_avg_psnr_vdsr_epoch > best_psnr_vdsr:
        best_psnr_vdsr = val_avg_psnr_vdsr_epoch
        torch.save(vdsr.state_dict(), 'best_vdsr.pth')
        print(f"Saved VDSR model with PSNR {best_psnr_vdsr:.4f}")

    print(f"Epoch [{epoch+1}/{num_epochs}] completed: WDSRA Loss: {avg_epoch_loss_srcnn:.4f}, PSNR: {avg_psnr_srcnn_epoch:.4f}, Validation PSNR: {val_avg_psnr_srcnn_epoch:.4f}")
    print(f"Epoch [{epoch+1}/{num_epochs}] completed: vdsr Loss: {avg_epoch_loss_vdsr:.4f}, PSNR: {avg_psnr_vdsr_epoch:.4f}, Validation PSNR: {val_avg_psnr_vdsr_epoch:.4f}")

    log_file.write(f"Epoch {epoch+1}: WDSRA PSNR: {avg_psnr_srcnn_epoch:.4f}, Validation PSNR: {val_avg_psnr_srcnn_epoch:.4f}\n")
    log_file.write(f"Epoch {epoch+1}: vdsr PSNR: {avg_psnr_vdsr_epoch:.4f}, Validation PSNR: {val_avg_psnr_vdsr_epoch:.4f}\n")

    # log_file.flush()

log_file.close()


Epoch 1/24:   0%|          | 0/12500 [00:00<?, ?batch/s]/home/robot/anaconda3/envs/pcb/lib/python3.12/site-packages/torch/optim/lr_scheduler.py:216: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn(
Epoch 1/24:   0%|          | 38/12500 [00:06<33:40,  6.17batch/s] 


KeyboardInterrupt: 

# 5. Testing

In [ ]:
# Tải và chuyển đổi ảnh
transform = transforms.ToTensor()
output_image_dir = 'output/images'
os.makedirs(output_image_dir, exist_ok = True)
# Duyệt qua các ảnh trong thư mục
lr_image_files = os.listdir(test_lr_dir)
hr_image_files = os.listdir(test_hr_dir)
lr_image_files.sort()
hr_image_files.sort()
psnr_values_sobel = []
psnr_values_canny = []


In [ ]:
!nvidia-smi
with torch.no_grad():
    for lr_image_file, hr_image_file in tqdm(zip(lr_image_files, hr_image_files), unit = 'batch'):
        # Đường dẫn đến ảnh
        lr_image_path = os.path.join(test_lr_dir, lr_image_file)
        hr_image_path = os.path.join(test_hr_dir, hr_image_file)
        output_image_path = os.path.join(output_image_dir, lr_image_file[:-4] + '_edsrx4.jpg')

        # Tải và chuyển đổi ảnh
        lr_image = Image.open(lr_image_path)
        hr_image = Image.open(hr_image_path)

        lr_image = transform(lr_image).unsqueeze(0).cuda()  # Thêm batch dimension và chuyển sang CPU
        hr_image = transform(hr_image).unsqueeze(0).cuda()  # Thêm batch dimension và chuyển sang CPU

        # Dự đoán
        sobel = sobelsr(lr_image)
#         canny = cannysr(lr_image)

        # Tính toán PSNR
        psnr_sobel = calculate_psnr(sobel, hr_image)
#         psnr_canny = calculate_psnr(canny, hr_image)

        psnr_values_sobel.append(psnr_sobel)
#         psnr_values_canny.append(psnr_canny)

#         # Chuyển đổi tensor đầu ra thành ảnh và lưu
#         output_image = output.squeeze(0).cuda()  # Loại bỏ batch dimension và chuyển tensor sang CPU
#         output_image = transforms.ToPILImage()(output_image)  # Chuyển tensor thành ảnh PIL
#         output_image.save(output_image_path)  # Lưu ảnh

# Tính toán PSNR trung bình
average_psnr_sobel = sum(psnr_values_sobel) / len(psnr_values_sobel)
print(f"Average PSNR sobel: {average_psnr_sobel:.2f} dB")

# average_psnr_canny = sum(psnr_values_canny) / len(psnr_values_canny)
# print(f"Average PSNR canny: {average_psnr_canny:.2f} dB")